In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("/content/drive/MyDrive/Mobile_Reviews_Rating/data_review_title.csv")
df.head()

,reviewtitle,rating
0,Honest review of an edm music lover\n,3
1,Unreliable earphones with high cost\n,1
2,Really good and durable.\n,4
3,stopped working in just 14 days\n,1
4,Just Awesome Wireless Headphone under 1000...😉\n,5


In [3]:
df.shape

(14337, 2)

In [4]:
 df.dtypes

reviewtitle    object
rating          int64
dtype: object

In [5]:
df.isnull().sum()

reviewtitle    0
rating         0
dtype: int64

In [6]:
df["reviewtitle"][0]

'Honest review of an edm music lover\n'

In [7]:
y = df["rating"]
y

0        3
1        1
2        4
3        1
4        5
        ..
14332    4
14333    5
14334    1
14335    5
14336    1
Name: rating, Length: 14337, dtype: int64

In [8]:
df.describe()

,rating
count,14337.000000
mean,3.675874
std,1.503409
min,1.000000
25%,3.000000
50%,4.000000
75%,5.000000
max,5.000000


In [9]:
reviews=df.copy()
reviews.head(10)

,reviewtitle,rating
0,Honest review of an edm music lover\n,3
1,Unreliable earphones with high cost\n,1
2,Really good and durable.\n,4
3,stopped working in just 14 days\n,1
4,Just Awesome Wireless Headphone under 1000...😉\n,5
5,Charging port not working\n,1
6,Loved the color and sound !!\n,4
7,Great sound with worst mic.\n,3
8,Battery life is good enough to keep going all ...,5
9,Very very bad Durabity\n,1


In [10]:
import re
re.sub("<.*?>","",reviews['reviewtitle'][0])

'Honest review of an edm music lover\n'

In [11]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [12]:
import time
start = time.time()
from IPython.display import clear_output
corpus = []
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
wordnet = WordNetLemmatizer()
for i in range(0,len(reviews)):
    print(i)
    clear_output(wait=True)
    data =  re.sub('[^a-zA-Z]', ' ', reviews["reviewtitle"][i])
    data = re.sub('<.*?>'," ",data)
    data = data.lower()
    data = data.split()
    data = [wordnet.lemmatize(word) for word in data if not word in stopwords.words('english')]
    data = ' '.join(data)
    corpus.append(data)
end = time.time()
print(end - start)
print(corpus)

52.16224551200867
['honest review edm music lover', 'unreliable earphone high cost', 'really good durable', 'stopped working day', 'awesome wireless headphone', 'charging port working', 'loved color sound', 'great sound worst mic', 'battery life good enough keep going day', 'bad durabity', 'running become comfortable', 'disappointed', 'extremely good bt earphone loud super bass', 'product good quality', 'expected th day purchase', 'good earphone good sound quality build quality', 'good sound bad designed', 'good product decent pro', 'average product buy', 'overall good worth', 'budget friendly', 'received product one side earphone working', 'satisfied totally', 'buy', 'unable connect two device simultaneously', 'read carefully', 'defective product', 'dont buy', 'really boat beating others price range k', 'good', 'really cheep quality', 'awesome bass battery life', 'review update product failed month', 'ok quality', 'good sound quality', 'amazing product go', 'frequent failure', 'puchas

In [13]:
print(len(corpus))

14337


In [14]:
import pickle

In [15]:
pickle.dump(corpus, open("corpus_mobile_rating.pkl", "wb"))

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
cv_2 = TfidfVectorizer(max_features=5000,ngram_range=(1,3))
X_tfid = cv_2.fit_transform(corpus)
X_tfid.shape

(14337, 5000)

In [17]:
pickle.dump(cv_2, open("cv_2_mobile_rating.pkl", "wb"))

In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_tfid, y, test_size=0.20, random_state=0)

from sklearn.tree import DecisionTreeRegressor
model = DecisionTreeRegressor().fit(X_train, y_train)

In [19]:
import numpy as np
from sklearn import metrics

y_pred=model.predict(X_test)

import sklearn.metrics as sm
print("Mean absolute error =", round(sm.mean_absolute_error(y_test, y_pred), 2)) 
print("Mean squared error =", round(sm.mean_squared_error(y_test,y_pred), 2)) 
print("Explain variance score =", round(sm.explained_variance_score(y_test, y_pred), 2)) 
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean absolute error = 0.79
Mean squared error = 1.45
Explain variance score = 0.36
Root Mean Squared Error: 1.2032213587446998


In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
wordnet = WordNetLemmatizer()
def new_review(new_review):
    new_review = new_review
    new_review = re.sub('[^a-zA-Z]', ' ', new_review)
    new_review = new_review.lower()
    new_review = new_review.split()
    all_stopwords = stopwords.words('english')
    all_stopwords.remove('not')
    new_review = [wordnet.lemmatize(word) for word in new_review if not word in set(all_stopwords)]
    new_review = ' '.join(new_review)
    new_corpus = [new_review]
    new_X_test = cv_2.transform(new_corpus).toarray()
    pred = model.predict(new_X_test)
    return pred

new_review = new_review(str(input("Enter new review...")))
print(new_review)

Enter new review...that was a quite nice one
[4.]


In [21]:
pickle.dump(model, open("model_mobile_rating.pkl", "wb"))